## pyESM - Python based Engineering Systems Modelling (pyESM)
Python based engineering systems modelling framework based on the Supply and Use structure

- - -
PACKAGE LOCAL INSTALLATION and USAGE

Editable local installation:
1. Create a virtual environment based on 'environment.yml' file.
2. In the cmd, run: >>> python -m pip install -e "path/to/pyesm"
3. From the virtual environment: >>> import esm
4. Use esm APIs (Model class)


- - -
CREATE/UPDATE ENVIRONMENT BASED ON YML FILE IN PROJECT REPO

In the prompt:
- From the path where environment.yml is present: >>> conda env create -f environment.yml
- The environment is named "esm", so type >>> conda activate esm

UPDATE ENVIRONMENT YML FILE (in case of modifications)

In the prompt: 
- activate the working environment: >>> conda activate your_environment_name
- export environment.yml file based on the working environment: >>> conda env export > environment.yml


- - - 
MODEL DIRECTORY GENERATION

Generation of a model directory based on a model template or with blank setup files.

In [ ]:
# create model directory with essential setup files
# to be used only to generate models from scratch
# template models can be imported 
import esm

model_dir_name = '3_sut_multi_year_rcot_cap'
main_dir_path = 'D:\git_repos\pyesm\default'

esm.create_model_dir(
    model_dir_name=model_dir_name,
    main_dir_path=main_dir_path,
    default_model=None,
    force_overwrite=False,
)

- - - 
GENERATION OF A NEW MODEL FROM SCRATCH

Generation of a new model defined by setup files.

Step-by-step model creation with sets, data and problems generation.

In [1]:
# generate model instance based on setup files filled by the user.
# validates model directory before executing.
import esm

model_dir_name = '3_sut_multi_year_rcot_cap'
main_dir_path = 'D:\git_repos\pyesm\default'

model = esm.Model(
    model_dir_name=model_dir_name,
    main_dir_path=main_dir_path,
    use_existing_data=False,
    log_level='info',
)

INFO | Model | 'Model' object initialization...
INFO | Model.file_manager | 'FileManager' object generated.
INFO | Model | Defining settings from model arguments.
INFO | Model | Defining paths from model arguments.
INFO | Model | Model directory and required setup files validated.
INFO | Model.core | 'Core' object initialization...
INFO | Model.core.sql_manager | 'SQLManager' object generation.
INFO | Model.core.index | 'Index' object initialization...
INFO | Model.core.index | Loading and validating data from file, generating 'SetTable' objects.
INFO | Model.core.index | Loading and validating data from file, generating 'DataTable' objects.
INFO | Model.core.index | Fetching and validating data, generating 'Variable' objects.
INFO | Model.core.index | 'Index' object initialized.
INFO | Model.core.database | 'Database' object initialization...
INFO | Model.core.database | Overwriting database 'database.db'
INFO | Model.core.database | Overwriting sets excel file 'sets.xlsx'
INFO | Mode

In [4]:
# after filling sets.xlsx file: 
# - loading model coordinates to Index
# - generating sqlite database tables for sets and variables
# (variables tables in sqlite database empty)
model.load_model_coosrdinates()
model.initialize_blank_database()

INFO | Model | Generating blank SQLite database and excel input files.
INFO | Model.core.database | Loading Sets to 'database.db'.
ERROR | Model.core.sql_manager | Passed DataFrame and SQLite table '_set_SCENARIOS' headers mismatch.


ValueError: Passed DataFrame and SQLite table '_set_SCENARIOS' headers mismatch.

In [ ]:
# after filling input_data file/s:
# - loading input data into sqlite database variables tables
# - initialize problem
model.load_data_files_to_database()
model.initialize_problems()

In [ ]:
model.initialize_problems()

- - - 
MODEL GENERATION FROM EXISTING DATA

Generation of a new model working with existing database and data input files.

In [ ]:
# generate model instance based on setup files filled by the user.
# validates model directory (in case user relies on existing directory). 
# parse settings and paths.
# loading model coordinates
# initializing numerical problem
import esm 

model_dir_name = '1_sut_multi_year'
main_dir_path = 'D:\git_repos\pyesm\default'

model = esm.Model(
    model_dir_name=model_dir_name,
    main_dir_path=main_dir_path,
    use_existing_data=True,
    log_level='info'
)

- - -
DATA and MODEL UPDATE, MODEL RUN 

updating SQLite database with new data, 
re-initializing numerical problem

solving numerical model, 
results export to sqlite database, 
generation of powerbi dataset

In [ ]:
# in case of modifications in input data files (not in sets),
# update database and problem
model.update_database_and_problem()

In [ ]:
# in case of modifications in symbolic problem,
# update problems dataframe and symbolic problem
model.initialize_problems()

In [ ]:
# solve numerical problems
model.run_model(verbose=False)

In [ ]:
# once model has successfully solved, load endogenous parameters data to 
# sqlite database and generate powerbi dataset.
model.load_results_to_database()
model.generate_pbi_report()

- - - 
TESTS

In [ ]:
for var_key, variable in model.core.index.variables.items():
    print(
        var_key, ' ',
        'shape:', variable.shape, ' ',
        'shape_size:', variable.shape_size, ' ',
        # 'is_square:', variable.is_square, ' ',
        # 'is_vector:', variable.is_vector, ' ',
        # 'dim_labels:', variable.dim_labels, ' ',
        # 'dim_items:', variable.dim_items, ' ',
    )


In [ ]:
model.core.index.variables

In [ ]:
model.core.index.sets.technologies

In [ ]:
model.core.problem.symbolic_problem

In [ ]:
model.core.problem.numeric_problems

In [ ]:
import cvxpy as cp
import numpy as np

# Problem data.
m = 5
n = 2

# Construct the problem.
x = cp.Variable((n,1))
A = cp.Constant(np.random.randn(m, n))
b = cp.Constant(1)


C = cp.Constant(np.random.randn(n, n))
I = cp.Constant(np.eye(n))

objective = cp.Minimize(cp.sum_squares(A @ x - b))
constraints = [0 <= x, x <= 1]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve(solver='GUROBI', verbose=False)
# The optimal value for x is stored in `x.value`.

x.value


In [ ]:
v_24 = cp.Variable((2,4))
c_24 = cp.Constant(np.ones((2,4)))
c_41 = cp.Constant(np.ones((4,1)))

constraints = [v_24 == c_24 + c_41.T]
prob = cp.Problem(cp.Minimize(1), constraints)
prob.solve(solver='GUROBI', verbose=False)

print(f'v_24 = {v_24.value} \n')
print(f'c_24 = {c_24.value} \n')
print(f'c_41 = {c_41.T.value} \n')


In [ ]:
# DEFINIZIONE COSTANTI PER TRASFORMARE I VARI OPERATORI VETTORIALI
import cvxpy as cp
import numpy as np

var_a = cp.Variable((3,2))
var_a.value = np.ones((var_a.shape))*3
var_b = cp.Variable((3,1))

i_21 = cp.Constant(np.ones((2,1)))
i_13 = cp.Constant(np.ones((1,3)))

allowed_vars = {
    'A': var_a,
    'B': var_b,
    'i_21': i_21,
    'i_13': i_13,
}

allowed_ops = {
    '+': '+',
    '-': '-',
    '*': '*',
    '@': '@',
    '==': '==',
    '>=': '>=',
    '<=': '<=',
    '(': '(',
    ')': ')',
    ',':',',
    'sum': cp.sum,
}

expr = model.core.problem.execute_cvxpy_code(
    expression='i_13 @ A @ i_21',
    allowed_variables=allowed_vars,
    allowed_operators=allowed_ops
)

expr2 = model.core.problem.execute_cvxpy_code(
    expression='sum(B)',
    allowed_variables=allowed_vars,
    allowed_operators=allowed_ops
)

expr

In [ ]:
model.core.database.sqltools.open_connection()

var = model.core.database.sqltools.filtered_table_to_dataframe(
    table_name='u',
    filters_dict={
        's_Name': ['Reference'],
        'dt_Name': [2023],
        'f_Name': ['Energy', 'Steel'],
        't_Name': ['PV power plant', 'Steel factory', 'Gas power plant']
    }
)

# filter_1 = test.core.database.sqltools.get_related_table_keys(
#     child_column_name='f_Name',
#     parent_table_name='_set_FLOWS',
#     parent_table_fields={
#         'f_Category': ['Product flow'],
#     }
# )

# filter_2 = test.core.database.sqltools.get_related_table_keys(
#     child_column_name='dt_Name',
#     parent_table_name='_set_DATETIME',
#     parent_table_fields={
#         'dt_Name': [2023],
#     }
# )

# var_custom_filter = test.core.database.sqltools.filtered_table_to_dataframe(
#     table_name='v',
#     filters_dict={
#         **filter_1,
#         **filter_2,
#         **{'f_Name': ['Steel']},
#     }
# )

model.core.database.sqltools.close_connection()

# filter_1, filter_2
var